# Imports

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install scikit-learn-extra

     |████████████████████████████████| 1.9MB 11.4MB/s 
     |████████████████████████████████| 3.2MB 42.4MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7367c65c690028597b07994dc4447fd1444e9e719b8dfb1ae7837190341223b3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 10.9MB/s 
     |████████████████████████████████| 245kB 21.1MB/s 
     |████████████████████████████████| 112kB 22.3MB/s 
     |████████████████████████████████| 1.2MB 10.9MB/s 
     |████████████████████████████████| 624kB 11.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.1.0b2-cp37-cp37m-linux_x86_64.whl size=339937 sha256=adaf4

In [ ]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import argparse
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import os
import random
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import sys
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.gaussian_process.kernels import RBF
import torch
from scipy.stats import multivariate_normal
import  scipy.stats as st
from matplotlib import cm
import torch.optim as optim
from __future__ import print_function
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, AutoModel
import nltk
from torch.utils.data import DataLoader
import pickle
import spacy
from gensim import corpora, models, similarities
import gensim
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# HotpotQA


In [ ]:
DISTRACTORS_PARS_LEN = 1

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", 'distractor')

Dataset hotpot_qa downloaded and prepared to /root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d. Subsequent calls will reuse this data.


In [ ]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

In [ ]:

def get_token_lenght(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    return len(inputs['input_ids'][0])

def remove_distractors(example):
    all_pars = example['context']['title']
    gold_pars = list(set(example['supporting_facts']['title']))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or DISTRACTORS_PARS_LEN == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), DISTRACTORS_PARS_LEN)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
    example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 
    return example

In [ ]:
train_dataset = train_dataset.map(remove_distractors)
validation_dataset = validation_dataset.map(remove_distractors)

In [ ]:
train_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 90447
})

In [ ]:
validation_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 7405
})

In [ ]:
def read_hotpot(hotpot_dataset):

    contexts = []
    questions = []
    answers = []
    all_contexts = hotpot_dataset['context'][:]
    all_questions = hotpot_dataset['question'][:]
    all_answers = hotpot_dataset['answer'][:]
    invalid_quests = 0
    for i in range(len(all_contexts)):

        answer = {}
        context = "".join(all_contexts[i]['sentences'])
        answer['text'] = all_answers[i]
        answer_start = context.find(answer['text'])
        if answer_start == -1:
            if answer['text'] not in ["yes", "no"]:
                invalid_quests += 1
            answer['answer_start'] = answer_start
            answers.append(answer)
            contexts.append(context)
            questions.append(all_questions[i])
        else:
            answer['answer_start'] = answer_start
            answers.append(answer)
            contexts.append(context)
            questions.append(all_questions[i])
    print(f'invalid percentage {invalid_quests/len(hotpot_dataset)*100:.2f}')
    return contexts, questions, answers

START_TEST = math.floor(len(validation_dataset)/2)
val_contexts, val_questions, val_answers = read_hotpot(validation_dataset)
teaching_contexts = val_contexts[:START_TEST]
teaching_questions = val_questions[:START_TEST]
teaching_answers = val_answers[:START_TEST]
validation_contexts = val_contexts[START_TEST:]
validation_questions = val_questions[START_TEST:]
validation_answers = val_answers[START_TEST:]


invalid percentage 0.00


In [ ]:
len(val_answers)

7405

## Utils

In [ ]:

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)



def evaluate_truth_pred(truths, preds):
    '''
    truths, preds: matched arrays of ground truth answers and predictions
    '''
    f1 = exact_match = total = 0
    for i in range(len(truths)):
        total += 1
        if truths[i] in ['yes', "no"]:
            continue
        ground_truths = [truths[i]]
        prediction = preds[i]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / (total+ 0.00000000001)
    f1 = 100.0 * f1 / (total+ 0.00000000001)

    return {'exact_match': exact_match, 'f1': f1}


def get_answer( model, tokenizer, context, question):
    # 1. TOKENIZE THE INPUT
    # note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for
    # exploration but you cannot feed that into a model.
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    inputs = inputs.to(device)
    # 2. OBTAIN MODEL SCORES
    # the AutoModelForQuestionAnswering class includes a span predictor on top of the model.
    # the model returns answer start and end scores for each word in the text
    answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
    # 3. GET THE ANSWER SPAN
    # once we have the most likely start and end tokens, we grab all the tokens between them
    # and convert tokens back to words!
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer
# https://huggingface.co/transformers/migration.html
def model_evaluation(model, tokenizer, questions, contexts, answers, to_print = False):
    preds = []
    my_list = list(range(len(questions)))
    with tqdm(total=len(my_list)) as pbar:
        for ex in range(len(questions)):
            answer = get_answer(model, tokenizer, contexts[ex],questions[ex])
            preds.append(answer)
            if ex % 100 == 0 and to_print:
                print("context " +contexts[ex] )
                print("quest " +questions[ex] )
                print("truth " +answers[ex]['text'] )
                print("pred " + answer)
            pbar.update(1)
    truths = [answers[i]['text'] for i in range(len(answers))]
    scores = evaluate_truth_pred(truths, preds)
    print(scores)
    return scores




# Teaching setup

## Human model Complex


In [ ]:
# downloads
spacy.load('en')
parser = English()
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# see https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#import pyLDAvis.gensim and https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def detect_question(token):
    # assume token is string
    tag = nltk.pos_tag([token])
    if tag[0][1] in ["WP", "WRB", "WP$"]:
        return True
    else:
        return False
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if (token not in en_stop) or  (detect_question(token))]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def learn_lda_model():
    file_name = "../squad/train-v2.0.json"
    paragraphs = []
    with open(file_name) as f:
        dataset = json.load(f)['data']
    for article in dataset:
        for paragraph in article['paragraphs']:
            paragraphs.append(paragraph['context'])

    # prep data
    paragraphs_lda = []
    for i in range(len(paragraphs)):
        paragraphs_lda.append(prepare_text_for_lda(paragraphs[i]))

    dictionary = corpora.Dictionary(paragraphs_lda)
    corpus = [dictionary.doc2bow(text) for text in paragraphs_lda]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    NUM_TOPICS = 10
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('lda_model_10_squad.gensim')
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

class LDA_model:
    def __init__(self):
        self.n_topics = 10
        self.model =  models.LdaModel.load('lda_model_10_squad.gensim')
        self.dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
        self.corpus = pickle.load(open('corpus.pkl', 'rb'))
    def predict(self, paragraph):
        paragraph = prepare_text_for_lda(paragraph)
        new_doc_bow = self.dictionary.doc2bow(paragraph)
        result = self.model.get_document_topics(new_doc_bow)
        vector = [0] * self.n_topics
        for top in result:
            vector[top[0]] = top[1]
        return vector
    def show_topics(self):
        topics = self.model.print_topics(num_words=10)
        for topic in topics:
            print(topic)
#learn_lda_model()


In [ ]:

#import pyLDAvis.gensim

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def detect_question(token):
    # assume token is string
    tag = nltk.pos_tag([token])
    if tag[0][1] in ["WP", "WRB", "WP$"]:
        return True
    else:
        return False



def question_classification(question):
    question_words = ["what", "when", "where", "which", "who", "whom", "whose", "why", "how"]
    question_vec = [0] * len(question_words)
    tokens = tokenize(question)
    tokens = [get_lemma(token) for token in tokens]
    all_zero = True
    for token in tokens:
        for i in range(len(question_words)):
            if token == question_words[i]:
                question_vec[i] == 1
                all_zero = False
    if all_zero:
        print(f' question: {question}')
    return question_vec, all_zero

def question_test():
    file_name = "../squad/dev-v2.0.json"
    paragraphs = []
    total = 0
    zeros = 0
    with open(file_name) as f:
        dataset = json.load(f)['data']
    for article in dataset:
        print(article["title"])
        for paragraph in article['paragraphs']:
            paragraphs.append(paragraph['context'])
            for qa in paragraph['qas']:
                total +=1
                a, b = question_classification(qa['question'])
                zeros += int(b)
    print(zeros/total*1.0)


## Human model Naiive

In [ ]:
class HumanPredictor_simple:
    def __init__(self, accuracy):
        '''
        accuracy and ai_accuracy: floats in [0,1]
        '''
        self.accuracy = accuracy
    def predict(self, contexts, questions, answers):
        '''
        expects array of strings for inputs
        returns list of string answer
        '''
        preds = []
        for i in range(len(questions)):
            coin = random.random() # random number between [0,1]
            if coin <= self.accuracy:
                preds.append(answers[i]['text'])
            else:
                # generate random answer
                sents = nltk.sent_tokenize(contexts[i])
                rand_sent = random.randint(0,len(sents)-1)
                preds.append(sents[rand_sent])
        return preds
    def prior_rejector(self, contexts, questions, answers):
        '''
        this rejector is agnostic to input, random deferall 1-self.accuracy times
        '''
        preds = []
        for i in range(len(questions)):
            coin = random.random() # random number between [0,1]
            preds.append(0)
            '''
            if coin >= self.accuracy:
                preds.append(1)
            else:
                preds.append(0)
            '''
        return preds

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
def kernel_similarity(x, y):
        kernel_dist = rbf_kernel(x.reshape(1, -1),y.reshape(1, -1))
        return kernel_dist[0][0]


## Our Approach

In [ ]:
class HumanLearner:
    def __init__(self, kernel):
        '''
        kernel: function that takes two inputs and returns a similarity
        prior rejector: returns rejector
        '''
        self.teaching_set = []
        self.kernel = kernel
    def predict(self, xs, prior_rejector_preds, to_print = False):
        '''
        xs: expected array of inputs
        '''
        preds = []
        idx = 0
        used_posterior = 0 
        if to_print:
            print("-- Human making reject predictions --")
            with tqdm(total=len(xs)) as pbar:
                for x in xs:
                    ball_at_x = []
                    similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                    for i in range(len(self.teaching_set)):
                        similarity = similarities[i]
                        if similarity >=  self.teaching_set[i][2]:
                            ball_at_x.append((self.teaching_set[i], similarity))
                    if len(ball_at_x) == 0: 
                        # use prior rejector
                        preds.append(prior_rejector_preds[idx])
                    else:
                        ball_at_x= sorted(ball_at_x, key=lambda tup: tup[1])
                        ball_at_x.reverse()
                        ball_at_x = ball_at_x[:10]
                        used_posterior += 1
                        ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0][0] for kk in range(len(ball_at_x))]))[0]
                        #normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                        normalization = len(ball_at_x)
                        #score_one = np.sum([ball_similarities[i]*ball_at_x[i][0][1] for i in range(len(ball_at_x))])
                        score_one = np.sum([ball_at_x[i][0][1] for i in range(len(ball_at_x))])
                        pred = score_one / normalization
                        if pred >= 0.5:
                            preds.append(1)
                        else:
                            preds.append(0)
                    idx += 1
                    pbar.update(1)
        else:
            for x in xs:
                ball_at_x = []
                similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                for i in range(len(self.teaching_set)):
                    similarity = similarities[i]
                    if similarity >=  self.teaching_set[i][2]:
                        ball_at_x.append(self.teaching_set[i])
                if len(ball_at_x) == 0: 
                    # use prior rejector
                    preds.append(prior_rejector_preds[idx])
                else:
                    used_posterior += 1
                    ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0] for kk in range(len(ball_at_x))]))[0]
                    normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                    score_one = np.sum([ball_similarities[i]*ball_at_x[i][1] for i in range(len(ball_at_x))])
                    pred = score_one / normalization
                    if pred >= 0.5:
                        preds.append(1)
                    else:
                        preds.append(0)
                idx += 1
        if to_print:
            print(f'Used posterior {used_posterior/len(xs)*100:.2f}')
        return preds

    def add_to_teaching(self, teaching_example):
        '''
        teaching_example: (x, label, gamma)
        '''
        self.teaching_set.append(teaching_example)

    def remove_last_teaching_item(self):
        self.teaching_set = self.teaching_set[:-1]

In [ ]:
def compute_predictions_humanai(hum_preds, hum_rejector, ai_preds, data_x):
    '''
    hum_preds: array of human predictions
    ai_preds: array of AI predictions
    hum_rejector: HumanLearner
    data_x: array of inputs

    Returns array of final predictions and deferalls
    '''
    predictions = []
    with torch.no_grad():
        reject_decisions = hum_rejector(data_x)
        for i in range(len(data_x)):
            if reject_decisions[i] == 1:
                # defer
                predictions.append(ai_preds[i])
            else:
                predictions.append(hum_preds[i])
    return predictions, reject_decisions

def get_metrics(preds, truths):
    # custom for each use case
    return evaluate_truth_pred(truths, preds)

def compute_metrics(human_preds, ai_preds, reject_decisions, truths, to_print = False):
    coverage = 1 - np.sum(reject_decisions)/len(reject_decisions)
    humanai_preds = []
    human_preds_sys = []
    truths_human = []
    ai_preds_sys = []
    truths_ai = []
    for i in range(len(reject_decisions)):
        if reject_decisions[i] == 1:
            humanai_preds.append(ai_preds[i])
            ai_preds_sys.append(ai_preds[i])
            truths_ai.append(truths[i])
        else:
            humanai_preds.append(human_preds[i])
            human_preds_sys.append(human_preds[i])
            truths_human.append(truths[i])
    humanai_metrics = get_metrics(humanai_preds, truths)
    human_metrics = get_metrics(human_preds_sys, truths_human)
    ai_metrics = get_metrics(ai_preds_sys, truths_ai)
    if to_print:
        print(f'Coverage is {coverage*100:.2f}')
        print(f' metrics of system are: {humanai_metrics}')
        print(f' metrics of human are: {human_metrics}')
        print(f' metrics of AI are: {ai_metrics}')
    return coverage, humanai_metrics, human_metrics, ai_metrics

In [ ]:
# Preliminary steps:
# Step 1.1: get predictions of human and predictions of AI on teaching set
# Step 1.2: get prior rejector decisions of Human on teaching set
# Step 2: Compute optimal deferal decision for each point in teaching set  
# Step 3: Compute optimal gamma for each point, can start with a constant guess

# Algorithms:
# Initialize HumanLearner
# Baseline 1:
# step 1: get k-mediods points
# step 2: add to HumanLearner
# step 3: evaluate on validation set

In [ ]:
# Get AI predictions
ai_teaching_preds = []
ai_validation_preds = []
ai_val_preds = []
ai_train_preds = []
with open('/content/predictions_ans_val.json', 'r') as handle:
    preds_val = json.load(handle)
#with open('preds_train_data.p', 'rb') as handle:
#    preds_train = pickle.load(handle)

for key, value in preds_val.items():
    ai_val_preds.append(value)

#for key, value in preds_train.items():
#    ai_train_preds.append(value)

ai_teaching_preds = ai_val_preds[:START_TEST]
ai_validation_preds = ai_val_preds[START_TEST:]


In [ ]:
# Get Human Predictions
human_predictor = HumanPredictor_simple(0.6)
hum_teaching_preds = []
hum_validation_preds = []
priorhum_teaching_preds = []
priorhum_validation_preds = []
print("--Getting Human predictions on teach and validation sets --\n")
hum_teaching_preds = human_predictor.predict(teaching_contexts, teaching_questions, teaching_answers)
hum_validation_preds = human_predictor.predict(validation_contexts, validation_questions, validation_answers)
print("--Getting prior rejector predictions on teach and validation sets") 
priorhum_teaching_preds = human_predictor.prior_rejector(teaching_contexts, teaching_questions, teaching_answers)
priorhum_validation_preds = human_predictor.prior_rejector(validation_contexts, validation_questions, validation_answers)

--Getting Human predictions on teach and validation sets --

--Getting prior rejector predictions on teach and validation sets


In [ ]:
# Optimal deferall decisions
opt_defer_teaching = []
opt_defer_validation = []
for ex in range(len(teaching_contexts)):
    f1_hum = metric_max_over_ground_truths(f1_score, teaching_answers[ex]['text'], [hum_teaching_preds[ex]])
    f1_ai = metric_max_over_ground_truths(f1_score, teaching_answers[ex]['text'], [ai_teaching_preds[ex]])

    if f1_ai > f1_hum:
        opt_defer_teaching.append(1)
    else:
        opt_defer_teaching.append(0)

for ex in range(len(validation_contexts)):
    f1_hum = metric_max_over_ground_truths(f1_score, validation_answers[ex]['text'], [hum_validation_preds[ex]])
    f1_ai = metric_max_over_ground_truths(f1_score, validation_answers[ex]['text'], [ai_validation_preds[ex]])
    if f1_ai > f1_hum:
        opt_defer_validation.append(1)
    else:
        opt_defer_validation.append(0)

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, priorhum_validation_preds, [validation_answers[i]['text'] for i in range(len(validation_answers))], True)

Coverage is 100.00
 metrics of system are: {'exact_match': 55.71158520118807, 'f1': 57.2702156863682}
 metrics of human are: {'exact_match': 55.71158520118807, 'f1': 57.2702156863682}
 metrics of AI are: {'exact_match': 0.0, 'f1': 0.0}


(1.0,
 {'exact_match': 55.71158520118807, 'f1': 57.2702156863682},
 {'exact_match': 55.71158520118807, 'f1': 57.2702156863682},
 {'exact_match': 0.0, 'f1': 0.0})

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, opt_defer_validation, [validation_answers[i]['text'] for i in range(len(validation_answers))])

(0.6594652984066973,
 {'exact_match': 80.82635700783126, 'f1': 86.47748080712576},
 {'exact_match': 84.47993447993413, 'f1': 84.69960534403612},
 {'exact_match': 73.75099127676388, 'f1': 89.9204402685561})

In [ ]:
import pickle
with open('/content/embeddings_data_val.p', 'rb') as handle:
    embeddings_val = pickle.load(handle)

embeddings_val = [embeddings_val[i][1] for i in range(len(embeddings_val))]
embeddings_val = np.asarray(embeddings_val)
teaching_embeddings = embeddings_val[:START_TEST]
validation_embeddings = embeddings_val[START_TEST:]


"\nwith open('embeddings_train_data.p', 'rb') as handle:\n    embeddings_train = pickle.load(handle)\nteaching_embeddings = [embeddings_train[i][1] for i in range(len(embeddings_train))]\nteaching_embeddings = np.asarray(teaching_embeddings)\nteaching_embeddings = np.asarray([np.mean(teaching_embeddings[i], axis = 0) for i in range(len(teaching_embeddings))])\n"

In [ ]:
# get optimal gammas
optimal_gammas = []
with tqdm(total=len(teaching_embeddings)) as pbar:
    for i in range(len(teaching_embeddings)):
        # get all similarities
        opt_defer_ex = opt_defer_teaching[i]
        similarities_embeds = rbf_kernel(teaching_embeddings[i].reshape(1,-1), np.asarray(teaching_embeddings))[0]
        sorted_sim = sorted([(similarities_embeds[k], opt_defer_teaching[k]) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])
        indicess = list(range(1, len(opt_defer_teaching)))
        indicess.reverse()
        for k in indicess:
            if sorted_sim[k][1] == opt_defer_ex and sorted_sim[k- 1][1] != opt_defer_ex:
                optimal_gammas.append(sorted_sim[k][0])
                break
        pbar.update(1)

100%|██████████| 3702/3702 [01:35<00:00, 38.59it/s]


In [ ]:
# Baseline get medoids
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=len(teaching_embeddings)-1, init='k-medoids++', max_iter=10000).fit(teaching_embeddings)
print(kmedoids.inertia_)
teaching_indices = kmedoids.medoid_indices_

0.7242298


In [ ]:
[teaching_contexts[i] for i in teaching_indices]

In [ ]:
human_learner = HumanLearner(kernel_similarity)


In [ ]:
for teach_ex_idx in teaching_indices:
    ex_embed = teaching_embeddings[teach_ex_idx]
    ex_label = opt_defer_teaching[teach_ex_idx]
    gamma = 0.95#optimal_gammas[teach_ex_idx] # random choice
    human_learner.add_to_teaching([ex_embed, ex_label, gamma])

In [ ]:
preds_teaching = human_learner.predict(teaching_embeddings, priorhum_teaching_preds, True)

  0%|          | 2/3702 [00:00<03:07, 19.71it/s]

-- Human making reject predictions --


100%|██████████| 3702/3702 [02:44<00:00, 22.51it/s]

Used posterior 100.00


In [ ]:
preds_val = human_learner.predict(validation_embeddings, priorhum_validation_preds, True)

  0%|          | 4/3703 [00:00<01:36, 38.14it/s]

-- Human making reject predictions --


100%|██████████| 3703/3703 [01:27<00:00, 42.43it/s]

Used posterior 100.00


In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, preds_val, [validation_answers[i]['text'] for i in range(len(validation_answers))], True)

Coverage is 87.90
 metrics of system are: {'exact_match': 56.79179044018348, 'f1': 59.857469325326164}
 metrics of human are: {'exact_match': 56.52841781874022, 'f1': 58.05541610373188}
 metrics of AI are: {'exact_match': 58.70535714285583, 'f1': 72.95051226347023}


(0.8790170132325141,
 {'exact_match': 56.79179044018348, 'f1': 59.857469325326164},
 {'exact_match': 56.52841781874022, 'f1': 58.05541610373188},
 {'exact_match': 58.70535714285583, 'f1': 72.95051226347023})

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, priorhum_validation_preds, [validation_answers[i]['text'] for i in range(len(validation_answers))], True)

Coverage is 100.00
 metrics of system are: {'exact_match': 56.19767755873601, 'f1': 57.734920668872135}
 metrics of human are: {'exact_match': 56.19767755873601, 'f1': 57.734920668872135}
 metrics of AI are: {'exact_match': 0.0, 'f1': 0.0}


(1.0,
 {'exact_match': 56.19767755873601, 'f1': 57.734920668872135},
 {'exact_match': 56.19767755873601, 'f1': 57.734920668872135},
 {'exact_match': 0.0, 'f1': 0.0})

In [ ]:
compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teaching, [teaching_answers[i]['text'] for i in range(len(teaching_answers))], True)

Coverage is 72.47
 metrics of system are: {'exact_match': 63.371150729335326, 'f1': 68.08218030799523}
 metrics of human are: {'exact_match': 60.79016026835609, 'f1': 62.07308123813874}
 metrics of AI are: {'exact_match': 70.1668302257108, 'f1': 83.90397893844079}


(0.7247433819556997,
 {'exact_match': 63.371150729335326, 'f1': 68.08218030799523},
 {'exact_match': 60.79016026835609, 'f1': 62.07308123813874},
 {'exact_match': 70.1668302257108, 'f1': 83.90397893844079})

In [ ]:
compute_metrics(hum_teaching_preds, ai_teaching_preds, priorhum_teaching_preds, [teaching_answers[i]['text'] for i in range(len(teaching_answers))], True)

Coverage is 100.00
 metrics of system are: {'exact_match': 54.80821177741746, 'f1': 56.437087734483555}
 metrics of human are: {'exact_match': 54.80821177741746, 'f1': 56.437087734483555}
 metrics of AI are: {'exact_match': 0.0, 'f1': 0.0}


(1.0,
 {'exact_match': 54.80821177741746, 'f1': 56.437087734483555},
 {'exact_match': 54.80821177741746, 'f1': 56.437087734483555},
 {'exact_match': 0.0, 'f1': 0.0})

## Our approach

In [ ]:
data_sizes  = []
indices_used = []
points_chosen = []


In [ ]:
MAX_SIZE = 2000
human_learner = HumanLearner(kernel_similarity)
errors = []
for itt in range(MAX_SIZE):
    print(f'New size {itt}')
    best_index = -1
    best_value = 0
    valid_indices = list(set(list(range(len(teaching_answers)))) - set(indices_used))
    subset_size = min(10, len(valid_indices))
    random_teach_subset = random.sample(valid_indices, subset_size) # used to take gradient steps
    random_validation_subset = random.sample(valid_indices, subset_size)
    print("")
    # for each point, add and see effect then remove
    counter_ = 0
    for j in random_teach_subset:
        counter_ += 1
        ex_embed = teaching_embeddings[j]
        ex_label = opt_defer_teaching[j]
        gamma = optimal_gammas[j] # random choice
        human_learner.add_to_teaching([ex_embed, ex_label, gamma])
        preds_teach = human_learner.predict([teaching_embeddings[i] for i in random_validation_subset], [priorhum_teaching_preds[i] for i in random_validation_subset])
        _, metrics, __, ___ = compute_metrics([hum_teaching_preds[kk] for kk in random_validation_subset], [ai_teaching_preds[kk] for kk in random_validation_subset], preds_teach, [teaching_answers[kk]['text'] for kk in random_validation_subset])
        #if counter_ % 100 == 0:
        #    print(metrics)
        acc = metrics["f1"]
        if acc >= best_value:
            best_value = acc
            best_index = j
        human_learner.remove_last_teaching_item()

    indices_used.append(best_index) # add found element to set used
    ex_embed = teaching_embeddings[best_index]
    ex_label = opt_defer_teaching[best_index]
    gamma = optimal_gammas[best_index] # random choice
    human_learner.add_to_teaching([ex_embed, ex_label, gamma])
    if itt % 50 == 0:
        print("####### Actual eval " +str(itt)+ " ###########")
        preds_teach = human_learner.predict(validation_embeddings, priorhum_validation_preds)
        _, metrics, __, ___ = compute_metrics(hum_validation_preds, ai_validation_preds, preds_teach, [validation_answers[i]['text'] for i in range(len(validation_answers))], True)
        errors.append(metrics)   
        print("##############################")